In [446]:
import requests
import pandas as pd
import numpy as np
import re
import random as rand
from IPython.core.display import HTML
import warnings
warnings.filterwarnings('ignore')

# API Sampling
I got more data from the API than artscrape.ipynb so that NA values wouldn't make as much of a difference in the EDA

In [447]:
# Get 40 random pages from the 165 available (165 pages, 25 stolen items per page)
# pages = [str(x) for x in rand.sample(range(1,166), 40)]

# Keep this set to aid with data cleaning and analysis consistency
pages_ex = ['61','161','81','125','115','19','55','51','150','137','64','98','108','154','9','145','121','50','33','128','4','162','78','99',
 '144','103','49','142','15','22','66','53','41','74','47','42','101','52','93','141']

In [448]:
# Get 25 stolen items per page from sample for 1000 total initially
params = [{"pageSize" : "25", "page" : x} for x in pages_ex]
base_url = "https://api.fbi.gov/@artcrimes"
rs = [requests.get(base_url, param) for param in params]

In [449]:
# Concatenate all json items dicts from sampled pages
thievery = pd.concat([pd.DataFrame(r.json()['items']) for r in rs], ignore_index=True)
# Keep only columns of interest
thievery = thievery.loc[:,['title', 'maker', 'crimeCategory', 'materials', 'period', 'measurements']]
display(HTML(thievery.head(20).to_html()))

,title,maker,crimeCategory,materials,period,measurements
0,Shakti II,Benjamin Creme,print,Lithograph,2005,"28"" x 21.75"""
1,Seated Woman,Raphael Soyer,"drawing,watercolour",Pastel on paper,1899-1987,28 x 22 in
2,Magda and Raymond,Walter Pach,paintings,Oil on canvas,1915,24 x 20 in
3,Jan,Edna Hibel,print,Lithograph,20th Century,None
4,Tiger painting,Michael Ward,paintings,Watercolor,20th Century,44 x 60 in
5,Pieta,Lucien Smith,paintings,Oil on canvas,20th Century,8 x 6.5 in
6,Beaded Blanket Strip,Ute | Sioux,"other,textiles",Brain tanned leather; sinew; multi-colored beads,1890s,70 x 5 in
7,December in Venice,Charles Polowetski,paintings,Oil on canvas,1937,Width: 30 in\r\n\r\nHeight: 24in.
8,Faun,Georg John Lober,sculpture,Bronze,None,14 in
9,Portrait of a Venetian Senator,Jacopo Tintoretto,paintings,Oil on canvas,1558-1594,47.50 x 36.50 cm


# Data Cleaning

## Maker
Since I have the least expertise with grouping artists, few adjustments were made. Semicolons separated either different creators of one piece or provided more information (Olmec; Mexico, for example). In the latter case, all information after the first semicolon was dropped.

In [450]:
# Combine unknown and anonymous into one category
thievery['maker'] = np.where(thievery['maker'].str.match('(unknown)|(anon.*?)', flags=re.IGNORECASE), "Unknown", thievery['maker'])

thievery['maker'].replace('Henry Klar; Gloria Klar', 'Henry and Gloria Klar', inplace=True)
thievery['maker'].replace('Faberge; Henrik Immanuel Wigstrom', 'Henrik Immanuel Wigstrom', inplace=True)
thievery['maker'].replace('Germany; Hans Ment', 'Hans Ment', inplace=True)
thievery['maker'].replace('Donald Pollard; Sidney Waugh', 'Donald Pollard and Sidney Waugh', inplace=True)
thievery['maker'].replace(r';.*', '', regex=True, inplace=True)

# Check for special cases (spelling, etc.)
# dict(thievery['maker'].value_counts())

## Crime Category
Crime was manually grouped according to my limited knowledge. Categories with 5 or fewer occurrences were moved to the "other" category

In [451]:
# Standardize description separation method
thievery['crimeCategory'].replace(',', '-', regex=True, inplace=True)

# Combine categories
thievery['crimeCategory'] = np.where(thievery['crimeCategory'].str.contains('instruments', regex=True), 'instruments', thievery['crimeCategory']) 
thievery['crimeCategory'] = np.where(thievery['crimeCategory'].str.contains('(jewelry)|(brooch)|(ring)|(earring)|(necklace)', regex=True), 'jewelry', thievery['crimeCategory'])
thievery['crimeCategory'] = np.where(thievery['crimeCategory'].str.contains('(rifle)|(blade)|(firearm)', regex=True), 'weapons', thievery['crimeCategory'])
thievery['crimeCategory'] = np.where(thievery['crimeCategory'].str.contains('figurine', regex=True), 'dolls-and-figurines', thievery['crimeCategory'])
thievery['crimeCategory'] = np.where(thievery['crimeCategory'].str.contains('mask', regex=True), 'clothing-and-costume', thievery['crimeCategory'])
thievery['crimeCategory'] = np.where(thievery['crimeCategory'].str.contains('(textile)|(tapestry)|(carpet)', regex=True), 'textiles', thievery['crimeCategory'])

# if five or fewer cases of category, move to "other"
def function_other(column):
    filter_material = thievery[f'{column}'].value_counts().to_frame().reset_index()
    other = list(filter_material[filter_material[f'{column}'] < 6]['index'])
    for i in other:
        thievery[f'{column}'] = np.where(thievery[f'{column}'] == i, 'other', thievery[f'{column}'])


function_other('crimeCategory')

# Check value counts for other special cases
# dict(thievery['crimeCategory'].value_counts())

## Materials
Materials went through a similar process as crimeCategory with one major change. The first material listed, if more than one was included in the piece, was considered the primary material, while the others didn't appear in the dataset. This aided with grouping. Then, the number of materials used for each piece was added to the dataset.

In [452]:
# Standardize to lowercase
thievery['materials'] = thievery['materials'].str.lower()

# Standardize separation of materials (Oil; Canvas instead of Oil On Canvas, etc.)
thievery['materials'] = thievery['materials'].str.replace(r'\s?((on)|(and)|\,|(with)|;)\s', ';', regex=True)

# Split into dataframe of materials
material_split = thievery['materials'].str.split(';', expand=True)

# Assume that the primary material would be listed first (not the best way but better for grouping)
thievery['primaryMaterial'] = material_split[0]
# Number of materials per piece
thievery['numMaterials'] = material_split.notna().sum(axis=1).astype(int)

# Combine categories
categories = ['wood','lithograph','oil','watercolor','plaster','glass','bronze','silver','gold','paper','bone','silk','ceramic','porcelain','gouache','aluminum','tortoise',
'etching','stone','serigraph','metal','aquatint','leather','quartz','crystal','tempera']

for category in categories:
    thievery['primaryMaterial'] = np.where(thievery['primaryMaterial'].str.contains(f'{category}', regex=True, flags=re.IGNORECASE), f'{category}', thievery['primaryMaterial'])

# measurement in material category, move
thievery['measurements'] = np.where(thievery['primaryMaterial'] == '60 X 48 In', thievery['primaryMaterial'], thievery['measurements'])

# if only one case of category, move to "Other"
function_other('primaryMaterial')

# Check for special cases
# dict(thievery['primaryMaterial'].value_counts())

## Period
Period was changed so that all dates were coded as century

In [453]:
# 1. Standardize to lowercase
thievery['period'] = thievery['period'].str.lower()

# 2. Take out qualifiers within century (mid, late, early)
thievery['period'] = thievery['period'].str.replace(r'(mid\-?\s?)|(late\s)|(early\s)|(second\shalf\sof\s)', '', regex=True)

# 3. Move misc. and date ranges that cover multiple centuries to other category
# a. Special case
thievery['period'] = np.where(thievery['period'] == '125-128 a.d.', '2nd century', thievery['period'])

# b. Extract centuries from ranges to dataframe (e.g., 1850-1970 would return 18 and 19)
ranges = thievery['period'].str.extract(r'([0-9]{2}).*\-([0-9]{2}).*')

# c. For each range, return "__th century" if the range is within one century, send to other category if not
thievery['period'] = np.where(thievery['period'].str.contains('\-'), \
    np.where(ranges[0]==ranges[1], ranges[0], 'other'), \
    thievery['period'])

# 4. Move years to their respective centuries (treat circa as exact year)
# For loop over 13th-20th centuries (before 1200 it's unlikely to have a specific year; 21st century has unique suffix)
for i in range(11, 20):
    thievery['period'] = np.where(thievery['period'].str.match('.*?'+ str(i) + '\d{2}'), str(i+1) + 'th century', thievery['period'])
# 21st century has different suffix
thievery['period'] = np.where(thievery['period'].str.match(r'.*20[0-9]{2}'), '21st century', thievery['period']) 

# Move measurements to measurements column
thievery['measurements'] = np.where(thievery['period'] == '14 x 10 in', thievery['period'], thievery['measurements'])
thievery['measurements'] = np.where(thievery['period'] == '30 in', thievery['period'], thievery['measurements'])

# Special cases (some hardcoding but oh well...), with BC specified but AD considered "default"
thievery['period'] = np.where(thievery['period'] == '600 ad', '6th century', thievery['period'])

# 5. Move remaining non-century periods to other (era like contemporary, coded incorrectly, etc.)
thievery['period'] = np.where(thievery['period'].str.contains('century'), thievery['period'], 'other')
function_other('period')

# 6. For less redundancy, move numbers + suffixes to century column
thievery['century'] = thievery['period'].replace("\scentury", "", regex=True)

# Check for special cases
# dict(thievery['period'].value_counts())
# dict(thievery['century'].value_counts())

## Measurements
Measurements was very complicated to clean, but essentially, I forced the qualitative coding of measurements inot a quantitative variable of the size of the piece and a categorical variable that reported whether the size was length, area, or volume.

In [454]:
# 1. Standardize to lowercase
thievery['measurements'] = thievery['measurements'].str.lower()

# Move values to period
thievery['measurements'].replace('early 17th century', '17th century', inplace=True)
thievery['period'] = np.where(thievery['measurements'] == 'early 17th century', thievery['measurements'], thievery['period'])

# 2. Special cases - far too many but oh well!
thievery['measurements'].replace('sheet size: 53.00 x 35.00 cm; print size: 35.00 x 27.00 cm', '53 x 35 cm', inplace=True)
thievery['measurements'].replace('13 3/4 x 17 1/2in. ; 35 x 44 1/2cm.', '13.75 x 17.5 in', inplace=True)
thievery['measurements'].replace('17 x 28in.', '17 x 28 in', inplace=True)
thievery['measurements'].replace('9.5 x 7 .5 in', '9.5 x 7.5 in', inplace=True)
thievery['measurements'].replace('approximately 7 inches in diameter', str(3.14159*((7/2)**2)) + ' x 1 in', inplace=True)
thievery['measurements'].replace('approximately 2.5 inches in diameter', str(3.14159*((2.5/2)**2)) + ' x 1 in', inplace=True)
thievery['measurements'].replace('panel: 16 x 15.25 in; image: 14.75 x 14.75 in', '16 x 15.25 in', inplace=True)
thievery['measurements'].replace('6 ft 7 in x 9 ft 10 in', '79 x 118 in', inplace=True)
thievery['measurements'].replace('11 x 8 ft', '132 x 96 in', inplace=True)
thievery['measurements'].replace('approximately 8 ft long', '96 in', inplace=True)
thievery['measurements'].replace("5' x 8'", '60 x 96 in', inplace=True)
thievery['measurements'].replace("7' x 4'", '84 x 48 in', inplace=True)
thievery['measurements'].replace("5' x 3'", '60 x 36 in', inplace=True)
thievery['measurements'].replace('height: 5.08; width: 2.51 cm; length: 2.46', '5.08 x 2.51 x 2.46 cm', inplace=True)
thievery['measurements'].replace("9' x 6'", '108 x 72 in', inplace=True)
thievery['measurements'].replace('14 1/2x23 1/2in.', '14.5 x 23.5 in', inplace=True)
thievery['measurements'].replace('height: 4 in; width: 5in', '4 x 5 in', inplace=True)
thievery['measurements'].replace('height: 4in.; width: 11in.', '4 x 11 in', inplace=True)
thievery['measurements'].replace('height: 5in; width: 3 in', '5 x 3 in', inplace=True)
thievery['measurements'].replace('height: 7.25in.width: 4.5in.', '7.25 x 4.5 in', inplace=True)
thievery['measurements'].replace('height: 5in.; width: 3.75 in.', '5 x 3.75 in', inplace=True)
thievery['measurements'].replace('height: 4in; width: 4.5in', '4 x 4.5 in', inplace=True)
thievery['measurements'].replace('height: 18.25in; width: 21.75in', '18.25 x 21.75 in', inplace=True)
thievery['measurements'].replace('1 5/8in. height 2 3/4in. length', '1.625 x 2.75 in', inplace=True)
thievery['measurements'].replace('height: 5 in\r\n\r\ndiameter: 4in.', '5 x ' + str(3.14159*((4/2)**2)) + ' x 1 in', inplace=True)
thievery['measurements'].replace('width: 30 in\r\n\r\nheight: 24in.', '30 x 24 in', inplace=True)
thievery['measurements'].replace('height: 31in;  width: 43in', '31 x 43 in', inplace=True)
thievery['measurements'].replace('height: 11.5in; width: 8.5in', '11.5 x 8.5 in', inplace=True)
thievery['measurements'].replace('height: 47in.; width: 53in.', '47 x 53 in', inplace=True)
thievery['measurements'].replace('sheet size', 'nan', inplace=True)
thievery['measurements'].replace('scroll', 'nan', inplace=True)
thievery['measurements'].replace('17th century', 'nan', inplace=True)
thievery['measurements'].replace('18.5 x 22', 'nan', inplace=True)
thievery['measurements'].replace('unframed dimensions: 12 3/4 x 11/78 in.; framed dimensions: 39 1/4 x 33 3/4 in.', '39.25 x 33.75 in', inplace=True)
thievery['measurements'].replace('height: 7 1/8 inches; width: 3-4 inches', '7.125 x 3.5 in', inplace=True)
thievery['measurements'].replace('not including frame', 'nan', inplace=True)
thievery['measurements'].replace('15 5/8 inches x 22 1/2 inches (39.7 cm x 57.2 cm)', '15.625 x 22.5 in', inplace=True)
thievery['measurements'].replace('base diameter: 2.25"; middle diameter: 3.5"; neck diameter: 1.5 "; top diameter: 3.5"', 'nan', inplace=True)
thievery['measurements'].replace('5" height x 4.5" square', '5 x 4.5 x 4.5 in', inplace=True)
thievery['measurements'].replace('unframed', 'nan', inplace=True)
thievery['measurements'].replace('paper', 'nan', inplace=True)
thievery['measurements'].replace('dia', 'nan', inplace=True)
thievery['measurements'].replace('19 5/8" x 24" (49.8 cm x 61 cm)', '19.625 x 24 in', inplace=True)
thievery['measurements'].replace('6" x 5" diam', '6 x ' + str(3.14159*((5/2)**2)) + ' x 1 in', inplace=True)
thievery['measurements'].replace('sheet size: 53 x 35 cm.; print size: 35 x 27 cm.', '53 x 35 cm', inplace=True)
thievery['measurements'].replace('dia: 6 in', str(3.14159*((6/2)**2)) + ' x 1 in', inplace=True)
thievery['measurements'].replace('19 x 24 in, frame: 26 x 31 in', '26 x 31 cm', inplace=True)
thievery['measurements'].replace('diameter: 2.25 in; 7 in x 3.75 in', '7 x 3.75 in', inplace=True)
thievery['measurements'].replace('16" x 16"; 40.64 cm x 40.64 cm', '16 x 16 in', inplace=True)
thievery['measurements'].replace('width: 10 in; height: 6 in; length: 95 in; weight: 350 lbs', '10 x 6 x 95 in', inplace=True)
thievery['measurements'].replace('24.1 x 30.9 cm; 23.4 x 30 cm', '24.1 x 30.9 cm', inplace=True)


# 3. Standardize how units are reported to "__ x __ x __ in/cm"
# a. Change " to in
thievery['measurements'].replace('\"', ' in', regex=True, inplace=True)

# b. Change __; __ to __ x __
thievery['measurements'].replace(';', ' x', regex=True, inplace=True)

# c. Remove words/phrases besides in/cm/etc.
thievery['measurements'].replace(r'((\sin\s)?depth(:\s)?)|((\sin\s)?height(:\s)?)|((\sin\s)?width(:\s)?)|((\sin\s)?length(:\s)?)|(about\s)|(\,?\s?frame:?\s?)|(\s\(with\sbase\))|(\s?\(?approximate(ly)?\)?)|(\,\sy:.*)|(\s\(shade\))', '', regex=True, inplace=True)

# d. Remove in and cm except at end of string
thievery['measurements'].replace(r'(in\.?|cm\.?)\sx', 'x', regex=True, inplace=True)

# 4. Replace fractions with decimals, case-by-case (not the best method but best I can do for now)
fracs = ['\s1/8','\s3/16','\s1/5','\s1/4','\s3/8', '\s7/16','\s1/2', '\s9/16', '\s3/4', '\s5/8', '\s11/16', '\s4/5','\s7/8']
decs = ['.125','.1875','.2','.25','.375', '.4375','.5', '.5625', '.75','.625', '.6875', '.8','.875']

for frac, dec in zip(fracs, decs):
    thievery['measurements'].replace(frac, dec, regex=True, inplace=True)

# 5. Change diameters to the respective number for area of a circle
# Get list of all diameters in column and convert to numeric area
diameters = list((3.14159*(thievery['measurements'].str.extract(r'diameter:\s([0-9]+\.?([0-9]+)?)')[0].astype(float)/2)**2).astype(str))
# add in 'x 1' to preserve dimension in later code (area instead of length)
diameters = [diameter + ' x 1' for diameter in diameters]
measurements = list(thievery['measurements'].copy())

for i in range(len(thievery)):
    # Filter out NoneType
    if isinstance(thievery.loc[i, 'measurements'], str):
        if thievery.loc[i, 'measurements'].find('diameter') != -1:
            # Sub out "Diameter: __" with diameter from list
            thievery.loc[i, 'measurements'] = re.sub(r'(diameter:\s[0-9]+(\.[0-9]+)?)', list(diameters)[i], measurements[i])

# 6. final adjustments
thievery['measurements'] = thievery['measurements'].str.lstrip()
thievery['measurements'].replace('34.5 in  x 6 in', '34.5 x 6 in', inplace=True)
thievery['measurements'].replace('14 x 12 x  10 in.', '14 x 12 x 10 in', inplace=True)
thievery['measurements'].replace('7.5 x 5 in, y', '7.5 x 5 in', inplace=True)
# dict(thievery['measurements'].value_counts())

# 7. Separate standardized measurements into numbers and words
m_split = thievery['measurements'].str.split(' ', expand=True)

# 8. Report whether measurement is length, area, or volume
# a. Get counts of not-nan values
counts = m_split.count(axis=1)

print(display(HTML(m_split.head(20).to_html())))

# b. 2  => __ in, i.e., one-dimensional or inches; 4 => __ x __ square; 6 => cubic
thievery['units'] = ['cubic feet' if i == 6 else 'square feet' if i == 4 else 'feet' if i == 2 else None for i in counts]

# 9. Report size of object
# a. Convert from cm to in as needed, all to ft
def cm_in_ft(i):
    col = m_split[i].astype(float)
    return np.where(m_split[1].str.match('cm.*') | m_split[3].str.match('cm.*') | m_split[5].str.match('cm.*'), round(col*0.393701/12,3), round(col/12,3))

# b. Convert to float, convert to feet instead of inches, and multiply across rows
m_split[0] = cm_in_ft(0)
m_split[2] = cm_in_ft(2)
m_split[4] = cm_in_ft(4)
m_split[6] = m_split[0].multiply(m_split[2], axis=0, fill_value=1).multiply(m_split[4], axis=0, fill_value=1)

# c. Set the piece's size
thievery['size'] = m_split[6]

,0,1,2,3,4,5
0,28,x,21.75,in,None,None
1,28,x,22,in,None,None
2,24,x,20,in,None,None
3,None,None,None,None,None,None
4,44,x,60,in,None,None
5,8,x,6.5,in,None,None
6,70,x,5,in,None,None
7,30,x,24,in,None,None
8,14,in,None,None,None,None
9,47.50,x,36.50,cm,None,None


None


# Final Data

In [455]:
# drop measurements column, finalize column order
thievery = thievery.loc[:,['title', 'maker', 'crimeCategory', 'primaryMaterial', 'numMaterials', 'century', 'size', 'units']]

In [456]:
# display first 20 rows of final table
display(HTML(thievery.head(20).to_html()))

,title,maker,crimeCategory,primaryMaterial,numMaterials,century,size,units
0,Shakti II,Benjamin Creme,print,lithograph,1,21st,4.227396,square feet
1,Seated Woman,Raphael Soyer,drawing-watercolour,other,2,other,4.276389,square feet
2,Magda and Raymond,Walter Pach,paintings,oil,2,20th,3.334000,square feet
3,Jan,Edna Hibel,print,lithograph,1,20th,NaN,None
4,Tiger painting,Michael Ward,paintings,watercolor,1,20th,18.335000,square feet
5,Pieta,Lucien Smith,paintings,oil,2,20th,0.361514,square feet
6,Beaded Blanket Strip,Ute | Sioux,textiles,other,3,19th,2.432361,square feet
7,December in Venice,Charles Polowetski,paintings,oil,2,20th,5.000000,square feet
8,Faun,Georg John Lober,sculpture,bronze,1,other,1.167000,feet
9,Portrait of a Venetian Senator,Jacopo Tintoretto,paintings,oil,2,other,1.866484,square feet


# Export to CSV

In [458]:
# thievery.to_csv("stolen_art_new.csv", index=False)